## Generating Benchmark Data for M5

The data is taken from the [M5 Forecasting - Accuracy](https://www.kaggle.com/c/m5-forecasting-accuracy) challenge on Kaggle, a competiton organized by The Makridakis Open Forecasting Center (MOFC) at the University of Nicosia ($50,000).

In [1]:
ROOT = "/data/cmu/large-scale-hts-reconciliation/"
data_dir = ROOT + "notebooks/"
TIME_HORIZON = 100

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet
from tqdm import tqdm
import pickle

Importing plotly failed. Interactive plots will not work.


### ETL (Extract, Transform & Load) from Kaggle

In [3]:
SMALL = False
df = pd.read_csv(data_dir + 'sales_train_evaluation.csv').fillna(0)
if SMALL: df = df.loc[df['cat_id'] == 'HOBBIES']
import collections
collections.Counter(df['dept_id'].values), \
collections.Counter(df['cat_id'].values), \
collections.Counter(df['store_id'].values), \
collections.Counter(df['state_id'].values)

(Counter({'HOBBIES_1': 4160,
          'HOBBIES_2': 1490,
          'HOUSEHOLD_1': 5320,
          'HOUSEHOLD_2': 5150,
          'FOODS_1': 2160,
          'FOODS_2': 3980,
          'FOODS_3': 8230}),
 Counter({'HOBBIES': 5650, 'HOUSEHOLD': 10470, 'FOODS': 14370}),
 Counter({'CA_1': 3049,
          'CA_2': 3049,
          'CA_3': 3049,
          'CA_4': 3049,
          'TX_1': 3049,
          'TX_2': 3049,
          'TX_3': 3049,
          'WI_1': 3049,
          'WI_2': 3049,
          'WI_3': 3049}),
 Counter({'CA': 12196, 'TX': 9147, 'WI': 9147}))

In [4]:
df

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,1,0,3,0,1,1,0,0,1,1
30486,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
30487,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,0,0,1,2,0,1,0,1,0,2
30488,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,1,1,1,4,6,0,1,1,1,0


### Compiling the hierarchies

In [5]:
try:
    df.drop(columns = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], inplace=True)
except:
    _  = 1

df['Cat/Dept/Item/State'] = df['id'].apply(lambda x: '_'.join(x.split('_')[:-1]))
df['Cat/Dept/Item'] = df['id'].apply(lambda x: '_'.join(x.split('_')[:-3]))
df['Cat/Dept'] = df['id'].apply(lambda x: '_'.join(x.split('_')[:-4]))
df['Cat'] = df['id'].apply(lambda x: x.split('_')[-6])

In [6]:
start = pd.to_datetime("2016-01-01")
ds = [start + pd.Timedelta(days = int(x[2:])) for x in (df.columns)[1:-4]]
ds = ['unique_id'] + ds + ['Cat/Dept/Item/State', 'Cat/Dept/Item', 'Cat/Dept', 'Cat']

In [7]:
df.columns = ds
df['unique_id'] = df['Cat/Dept/Item/State']

In [8]:
df

,unique_id,2016-01-02 00:00:00,2016-01-03 00:00:00,2016-01-04 00:00:00,2016-01-05 00:00:00,2016-01-06 00:00:00,2016-01-07 00:00:00,2016-01-08 00:00:00,2016-01-09 00:00:00,2016-01-10 00:00:00,...,2021-04-20 00:00:00,2021-04-21 00:00:00,2021-04-22 00:00:00,2021-04-23 00:00:00,2021-04-24 00:00:00,2021-04-25 00:00:00,Cat/Dept/Item/State,Cat/Dept/Item,Cat/Dept,Cat
0,HOBBIES_1_001_CA_1,0,0,0,0,0,0,0,0,0,...,0,0,3,3,0,1,HOBBIES_1_001_CA_1,HOBBIES_1_001,HOBBIES_1,HOBBIES
1,HOBBIES_1_002_CA_1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,HOBBIES_1_002_CA_1,HOBBIES_1_002,HOBBIES_1,HOBBIES
2,HOBBIES_1_003_CA_1,0,0,0,0,0,0,0,0,0,...,0,0,2,3,0,1,HOBBIES_1_003_CA_1,HOBBIES_1_003,HOBBIES_1,HOBBIES
3,HOBBIES_1_004_CA_1,0,0,0,0,0,0,0,0,0,...,0,1,3,0,2,6,HOBBIES_1_004_CA_1,HOBBIES_1_004,HOBBIES_1,HOBBIES
4,HOBBIES_1_005_CA_1,0,0,0,0,0,0,0,0,0,...,1,0,0,2,1,0,HOBBIES_1_005_CA_1,HOBBIES_1_005,HOBBIES_1,HOBBIES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3,0,0,2,2,0,3,1,4,1,...,1,1,0,0,1,1,FOODS_3_823_WI_3,FOODS_3_823,FOODS_3,FOODS
30486,FOODS_3_824_WI_3,0,0,0,0,0,5,0,1,1,...,0,0,1,0,1,0,FOODS_3_824_WI_3,FOODS_3_824,FOODS_3,FOODS
30487,FOODS_3_825_WI_3,0,6,0,2,2,4,1,8,5,...,0,1,0,1,0,2,FOODS_3_825_WI_3,FOODS_3_825,FOODS_3,FOODS
30488,FOODS_3_826_WI_3,0,0,0,0,0,0,0,0,0,...,6,0,1,1,1,0,FOODS_3_826_WI_3,FOODS_3_826,FOODS_3,FOODS


#### Roll-up to produce parent forecasts

In [9]:
tags = {}
tags['Cat'] = df['Cat'].unique()
tags['Cat/Dept'] = df['Cat/Dept'].unique()
tags['Cat/Dept/Item'] = df['Cat/Dept/Item'].unique()
tags['Cat/Dept/Item/State'] = df['Cat/Dept/Item/State'].unique()

In [10]:
tags

{'Cat': array(['HOBBIES', 'HOUSEHOLD', 'FOODS'], dtype=object),
 'Cat/Dept': array(['HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'FOODS_1',
        'FOODS_2', 'FOODS_3'], dtype=object),
 'Cat/Dept/Item': array(['HOBBIES_1_001', 'HOBBIES_1_002', 'HOBBIES_1_003', ...,
        'FOODS_3_825', 'FOODS_3_826', 'FOODS_3_827'], dtype=object),
 'Cat/Dept/Item/State': array(['HOBBIES_1_001_CA_1', 'HOBBIES_1_002_CA_1', 'HOBBIES_1_003_CA_1',
        ..., 'FOODS_3_825_WI_3', 'FOODS_3_826_WI_3', 'FOODS_3_827_WI_3'],
       dtype=object)}

In [11]:
node_map = {}
node_map_inv = {}

i = 0

for l in ['Cat/Dept/Item/State', 'Cat/Dept/Item', 'Cat/Dept', 'Cat']:
    for c in tags[l]:
        node_map[i] = c
        node_map_inv[c] = i
        i += 1

total_bottom = len(tags['Cat/Dept/Item/State'])
total_parent = len(tags['Cat/Dept/Item']) + len(tags['Cat/Dept']) + len(tags['Cat'])
parents = list(tags['Cat/Dept/Item']) + list(tags['Cat/Dept']) + list(tags['Cat'])
time_index = (df.columns)[1:-4]

In [12]:
pickle.dump([node_map_inv, node_map], open(data_dir + 'm5_node_map.pkl', 'wb'))

In [13]:
df

,unique_id,2016-01-02 00:00:00,2016-01-03 00:00:00,2016-01-04 00:00:00,2016-01-05 00:00:00,2016-01-06 00:00:00,2016-01-07 00:00:00,2016-01-08 00:00:00,2016-01-09 00:00:00,2016-01-10 00:00:00,...,2021-04-20 00:00:00,2021-04-21 00:00:00,2021-04-22 00:00:00,2021-04-23 00:00:00,2021-04-24 00:00:00,2021-04-25 00:00:00,Cat/Dept/Item/State,Cat/Dept/Item,Cat/Dept,Cat
0,HOBBIES_1_001_CA_1,0,0,0,0,0,0,0,0,0,...,0,0,3,3,0,1,HOBBIES_1_001_CA_1,HOBBIES_1_001,HOBBIES_1,HOBBIES
1,HOBBIES_1_002_CA_1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,HOBBIES_1_002_CA_1,HOBBIES_1_002,HOBBIES_1,HOBBIES
2,HOBBIES_1_003_CA_1,0,0,0,0,0,0,0,0,0,...,0,0,2,3,0,1,HOBBIES_1_003_CA_1,HOBBIES_1_003,HOBBIES_1,HOBBIES
3,HOBBIES_1_004_CA_1,0,0,0,0,0,0,0,0,0,...,0,1,3,0,2,6,HOBBIES_1_004_CA_1,HOBBIES_1_004,HOBBIES_1,HOBBIES
4,HOBBIES_1_005_CA_1,0,0,0,0,0,0,0,0,0,...,1,0,0,2,1,0,HOBBIES_1_005_CA_1,HOBBIES_1_005,HOBBIES_1,HOBBIES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3,0,0,2,2,0,3,1,4,1,...,1,1,0,0,1,1,FOODS_3_823_WI_3,FOODS_3_823,FOODS_3,FOODS
30486,FOODS_3_824_WI_3,0,0,0,0,0,5,0,1,1,...,0,0,1,0,1,0,FOODS_3_824_WI_3,FOODS_3_824,FOODS_3,FOODS
30487,FOODS_3_825_WI_3,0,6,0,2,2,4,1,8,5,...,0,1,0,1,0,2,FOODS_3_825_WI_3,FOODS_3_825,FOODS_3,FOODS
30488,FOODS_3_826_WI_3,0,0,0,0,0,0,0,0,0,...,6,0,1,1,1,0,FOODS_3_826_WI_3,FOODS_3_826,FOODS_3,FOODS


In [ ]:
parent_historical = np.zeros((total_parent, len(time_index))).astype(np.int64)

for _, ro in tqdm(df.iterrows()):
    vals = ro[time_index].values.astype(np.int64)
    parent_historical[node_map_inv[ro['Cat/Dept/Item']] - total_bottom, :] += vals
    parent_historical[node_map_inv[ro['Cat/Dept']] - total_bottom, :] += vals
    parent_historical[node_map_inv[ro['Cat']] - total_bottom, :] += vals

27662it [00:54, 517.07it/s]

In [ ]:
parent_df = pd.DataFrame(data = parent_historical)
parent_df.columns = time_index

In [ ]:
parent_df['unique_id'] = parents

In [ ]:
parent_df['Cat/Dept/Item/State'] = np.nan

In [ ]:
def p0(x):
    l = x.split('_')
    if len(l) < 3: return np.nan
    return x

def p1(x):
    l = x.split('_')
    if len(l) < 2: return np.nan
    return '_'.join(l[:2])

def p2(x):
    l = x.split('_')
    #if len(l) < 1: return np.nan
    return '_'.join(l[:1])

parent_df['Cat/Dept/Item'] = parent_df['unique_id'].apply(p0)
parent_df['Cat/Dept'] = parent_df['unique_id'].apply(p1)
parent_df['Cat'] = parent_df['unique_id'].apply(p2)
parent_df

In [ ]:
total_df = pd.concat([df, parent_df])
total_df.to_csv(data_dir + 'm5_historical.csv', index=False)

#### Compute edge list for LHTS

In [ ]:
edge_list = []
parent_list = []

for i, ro in tqdm(total_df.iterrows()):
    if type(ro['Cat/Dept/Item/State']) is str:
        edge_list.append([node_map_inv[ro['unique_id']], node_map_inv[ro['Cat/Dept/Item']]])
        parent_list.append([node_map_inv[ro['unique_id']], 
                            node_map_inv[ro['Cat/Dept/Item']],
                            node_map_inv[ro['Cat/Dept']],
                            node_map_inv[ro['Cat']],
                            ])
    elif type(ro['Cat/Dept/Item']) is str:
        edge_list.append([node_map_inv[ro['unique_id']], node_map_inv[ro['Cat/Dept']]])
        parent_list.append([node_map_inv[ro['unique_id']], 
                            node_map_inv[ro['Cat/Dept']],
                            node_map_inv[ro['Cat']],
                            -1
                            ])
    elif type(ro['Cat/Dept']) is str:
        edge_list.append([node_map_inv[ro['unique_id']], node_map_inv[ro['Cat']]])
        parent_list.append([node_map_inv[ro['unique_id']], 
                            node_map_inv[ro['Cat']],
                            -1,
                            -1
                            ])
    else:
        parent_list.append([node_map_inv[ro['unique_id']], 
                            -1, -1, -1])

In [ ]:
np.save(open(data_dir + 'm5_hierarchy.npy', 'wb'), np.array(edge_list))
np.save(open(data_dir + 'm5_prediction_raw/hierarchy.npy', 'wb'), np.array(edge_list))
np.save(open(data_dir + 'm5_hierarchy_parent.npy', 'wb'), np.array(parent_list))

In [ ]:
np.array(parent_list).shape

### Produce forecast

In [ ]:
FORECAST = False

if FORECAST:
    import dask.dataframe as dd

    ddf = dd.from_pandas(total_df, npartitions=4096)

    def predict(row):
        data = pd.DataFrame({'ds': (row.index)[1:-4][-TIME_HORIZON:], 'y':(row.values)[1:-4][-TIME_HORIZON:]})
        m = Prophet()
        m.fit(data)

        future = m.make_future_dataframe(periods=TIME_HORIZON)
        forecast = m.predict(future)

        return row.unique_id, forecast[['yhat']][-TIME_HORIZON:].values.reshape(-1)

    dask_series = ddf.apply(predict, axis=1, meta=('float', 'object'))  
    result = dask_series.compute(scheduler='processes')
    pickle.dump(result, open(data_dir + 'm5_prophet_forecast.pkl', 'wb'))

### Sanity check by plotting

In [ ]:
import pickle

if FORECAST:
    predictions = result
else:
    predictions = pickle.load(open(data_dir + 'm5_prophet_forecast_large.pkl', 'rb'))

In [ ]:
predictions

In [ ]:
import seaborn as sns

def smape(a, f):
    return 1/len(a) * np.sum(2 * np.abs(f-a) / (np.abs(a) + np.abs(f))*100)

pred = predictions.values[0][1]
gt = total_df.loc[total_df['unique_id'] == predictions.values[0][0]].values[0, 1:-4][-TIME_HORIZON:]

sns.lineplot(x=time_index[-TIME_HORIZON:], y=gt, label='ground truth')
sns.lineplot(x=time_index[-TIME_HORIZON:], y=pred, label='prediction')

print("SMAPE: ", smape(gt, pred))

In [ ]:
predictions.values[1]

### Export LHTS format

In [ ]:
pickle.dump(time_index, open(data_dir + "m5_time_index_all.pkl", "wb"))

In [ ]:
#parent_list

In [ ]:
total_ts = total_bottom + total_parent

pred_tensor = np.zeros((total_ts, TIME_HORIZON))
gt_tensor = np.zeros((total_ts, TIME_HORIZON))

for i, ro in tqdm(total_df.iterrows()):
    gt_tensor[node_map_inv[ro.unique_id], :] = ro.values[1:-4][-TIME_HORIZON:]
    
for unique_id, pred in tqdm(predictions):
    pred_tensor[node_map_inv[unique_id], :] = pred

In [ ]:
top_down_tensor = np.zeros((total_ts, TIME_HORIZON))
for l in tqdm(parent_list):
    i = l[0]
    for c in [l[3], l[2], l[1]]:
        if c != -1: break
    top_down_tensor[i, :] = gt_tensor[i, :] / (gt_tensor[c, :] + 1e-9)

In [ ]:
level_2_tensor = np.zeros((total_ts, TIME_HORIZON))

for l in tqdm(parent_list):
    nz = list(filter(lambda x: x!=-1, l))
    if len(nz) == 1: continue
    i = l[0]
    c = l[-2]
    level_2_tensor[i, :] = gt_tensor[i, :] / (gt_tensor[c, :] + 1e-9)
    
level_3_tensor = np.zeros((total_ts, TIME_HORIZON))

for l in tqdm(parent_list):
    nz = list(filter(lambda x: x!=-1, l))
    if len(nz) <= 2: continue
    i = l[0]
    c = l[-3]
    level_3_tensor[i, :] = gt_tensor[i, :] / (gt_tensor[c, :] + 1e-9)

In [ ]:
np.save(open(data_dir + 'm5_prediction_raw/pred_tensor.npy', 'wb'), pred_tensor)
np.save(open(data_dir + 'm5_prediction_raw/gt_tensor.npy', 'wb'), gt_tensor)
np.save(open(data_dir + 'm5_prediction_raw/top_down_tensor.npy', 'wb'), top_down_tensor)
np.save(open(data_dir + 'm5_prediction_raw/level_2_tensor.npy', 'wb'), level_2_tensor)
np.save(open(data_dir + 'm5_prediction_raw/level_3_tensor.npy', 'wb'), level_3_tensor)

In [ ]:
pred_tensor.shape

In [ ]:
N_CHUNKS = 12

for t, n in [(pred_tensor, 'pred_tensor'),
          (gt_tensor, 'gt_tensor'),
          (top_down_tensor, 'top_down_tensor'),
          (level_2_tensor, 'level_2_tensor'),
          (level_3_tensor, 'level_3_tensor')
         ]:
    l = np.array_split(t, N_CHUNKS)
    for i, p in enumerate(l):
        np.save(open(data_dir + 'm5_prediction_raw/mpi/'+ n + '_' + str(i) + '.npy', 'wb'), p)

### Export Nixtla format

In [ ]:
pickle.dump(tags, open('m5_prediction_raw/tags.pkl', 'wb'))

In [ ]:
y_gt = {'unique_id': [], 'ds': [], 'y': []}
yhat = {'unique_id': [], 'ds': [], 'y': []}

idx = time_index[-TIME_HORIZON:]

for _, ro in tqdm(total_df.iterrows()):
    vals = ro[time_index].values.astype(np.int64)[-TIME_HORIZON:]
    for i in range(TIME_HORIZON):
        y_gt['unique_id'].append(ro.unique_id)
        y_gt['ds'].append(idx[i])
        y_gt['y'].append(vals[i])
        
for unique_id, pred in tqdm(predictions):
    for i in range(TIME_HORIZON):
        yhat['unique_id'].append(unique_id)
        yhat['ds'].append(idx[i])
        yhat['y'].append(pred[i])

In [ ]:
Y_gt = pd.DataFrame(y_gt)
Y_hat = pd.DataFrame(yhat)
Y_hat.rename(columns = {'y': 'prophet'}, inplace=True)

Y_gt.to_csv(data_dir + 'm5_prediction_raw/nixtla_y_gt.csv', index=False)
Y_hat.to_csv(data_dir + 'm5_prediction_raw/nixtla_y_hat.csv', index=False)

In [ ]:
import gc
nrow = len(tags['Cat']) + len(tags['Cat/Dept']) + len(tags['Cat/Dept/Item']) + len(tags['Cat/Dept/Item/State'])
ncol = len(tags['Cat/Dept/Item/State'])

columns = list(tags['Cat/Dept/Item/State'])
rows = list(tags['Cat']) + list(tags['Cat/Dept']) + list(tags['Cat/Dept/Item']) + list(tags['Cat/Dept/Item/State'])

In [ ]:
raw_S = np.zeros((nrow, ncol))

In [ ]:
rows_map = {v:k for k,v in dict(enumerate(rows)).items()}
columns_map = {v:k for k,v in dict(enumerate(columns)).items()}

In [ ]:
for i, ro in tqdm(total_df.iterrows(), total=len(total_df)):
    if ro.unique_id not in columns_map: continue
    co = columns_map[ro.unique_id]
    ro4 = ro['Cat/Dept/Item/State']
    ro3 = ro['Cat/Dept/Item']
    ro2 = ro['Cat/Dept']
    ro1 = ro['Cat']
    if type(ro4) is str:
        raw_S[rows_map[ro4], co] = 1
    if type(ro3) is str:
        raw_S[rows_map[ro3], co] = 1
    if type(ro2) is str:
        raw_S[rows_map[ro2], co] = 1
    if type(ro1) is str:
        raw_S[rows_map[ro1], co] = 1

In [ ]:
raw_S.shape, pred_tensor.shape

In [ ]:
np.save(open(data_dir + 'm5_prediction_raw/S_tensor.npy', 'wb'), raw_S)

In [ ]:
S_df = pd.DataFrame(data=raw_S, dtype=np.int8)
S_df.columns = columns
S_df.index = rows

In [ ]:
pickle.dump(S_df, open('m5_prediction_raw/nixtla_s.pkl', 'wb'))

In [ ]:
S_df